##### Define Classification ResNet Model Structures

Build ResNet Model

###### [model_size] PyTorch's torchvision library includes pre-trained versions of the ResNet model. By defining this variable you can load different variants like ResNet18, ResNet50, and ResNet101<br>[fine_tune] This loop iterates over all parameters in the ResNet50 model and sets requires_grad to False. By doing so, these parameters will not be updated during training. This is known as "freezing" the model and is common when fine-tuning a pre-trained model on a relatively small new dataset. It helps prevent overfitting by not allowing the learned features from ImageNet to be modified too much.

In [1]:
import torchvision.models as models
import torch
import os

os.environ['TORCH_HOME'] = '../MODELs'

def build_binary_classification_resnet_model(model_size = 'resnet18', fine_tune = False):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    if model_size == 'resnet18':
        model = models.resnet18(pretrained=True)
    elif model_size == 'resnet50':
        model = models.resnet50(pretrained=True)
    elif model_size == 'resnet101':
        model = models.resnet101(pretrained=True)
    else:
        model = models.resnet18(pretrained=True)
    
    if fine_tune:
        for param in model.parameters():
            param.requires_grad = False
    
    num_features = model.fc.in_features
    model.fc = torch.nn.Sequential(
        torch.nn.Linear(num_features, 1),
        torch.nn.Sigmoid()
    )
    return model.to(device)     

k:\GIT\CCTV-object-alerts\LOCAL_ENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Get Training Set and Testing Set Ready

###### [Transforms] Ensure all images are the same size and format for the model. Normalization parameters are standard for models trained on ImageNet.<br>[ImageFolder] Loads images and labels from a directory structure where each subdirectory represents a class.<br>[random_split] Splits the dataset into training and testing sets.<br>[DataLoader] Batches, shuffles, and loads the data in parallel using worker processes.

In [2]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from collections import Counter

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageFolder(root='../DATA', transform=transform)

train_size = int(0.8 * len(dataset))  # 80% of the dataset for training
test_size = len(dataset) - train_size  # 20% of the dataset for testing
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class_counts = Counter()
for _, label in dataset:
    class_counts[label] += 1
label_mapping = {0: 'negative', 1: 'positive'}
for label, count in class_counts.items():
    print(f'{label_mapping[label]} (label={label}): {count} images')

negative (label=0): 1031 images
positive (label=1): 936 images


Get Training and Evaluation Functions Ready for ResNet Model

###### [Training]Here’s how you can set up a basic training loop to iterate through the training data, compute the loss, and update the model parameters.<br>[Evaluation]After training, it's important to evaluate your model on the test set to see how well it generalizes. This step involves calculating the accuracy or other relevant metrics without updating the model parameters.

In [3]:
def train_model(model, train_loader, optimizer, loss_function, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device).float().view(-1, 1)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_function(outputs, labels)
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize the model
            
            running_loss += loss.item()
        
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

In [4]:
def evaluate_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float().view(-1, 1)
            outputs = model(images)
            predicted = outputs.round()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on the test set: {accuracy:.2f}%')

Compute Accuracy over the entire ./DATA folder

###### [predict_image] Preprocesses an image and gets a prediction from the model.<br>[compute_accuracy] Loops through the "positive" and "negative" folders, uses the model to predict the label for each image, and computes the accuracy.

In [5]:
def predict_image(model, image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        output = model(image)
        prediction = output.round().item()
    return prediction
def compute_accuracy(model, data_dir):
    correct_predictions = 0
    total_images = 0

    for label, folder in enumerate(['negative', 'positive']):
        folder_path = os.path.join(data_dir, folder)
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            prediction = predict_image(model, image_path)
            if prediction == label:
                correct_predictions += 1
            total_images += 1

    accuracy = correct_predictions / total_images
    print(f'Accuracy: {accuracy:.2%}')

Compute Prediction over Random picked Image from ./DATA

In [6]:
import os
import random
from PIL import Image

def random_predict_image(model, data_dir, transform):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    folder = random.choice(['positive', 'negative'])
    folder_path = os.path.join(data_dir, folder)

    image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    if not image_files:
        print(f"No images found in {folder_path}")
        return

    image_file = random.choice(image_files)
    image_path = os.path.join(folder_path, image_file)

    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(image)
        prediction = output.round().item()

    actual_label = 1 if folder == 'positive' else 0
    print(f"Picked image: {image_path}")
    print(f"Actual label: {folder} (label={actual_label})")
    print(f"Predicted label: {prediction}")

##### Define Model

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

model = build_binary_classification_resnet_model()

loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

k:\GIT\CCTV-object-alerts\LOCAL_ENV\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
k:\GIT\CCTV-object-alerts\LOCAL_ENV\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


##### Load Model

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load('../MODELs/officer_classificaiton_resnet18_weights.pth'))
model = model.to(device)

##### Train and Save Model

In [10]:
train_model(model, train_loader, optimizer, loss_function, num_epochs=2)
evaluate_model(model, test_loader)
torch.save(model.state_dict(), '../MODELs/officer_classificaiton_resnet18_weights.pth')

Epoch 1, Loss: 0.021849732173141092
Epoch 2, Loss: 0.016701107685803435
Accuracy on the test set: 96.70%


##### Eval Model

In [11]:
compute_accuracy(model, '../DATA')

Accuracy: 99.29%


In [11]:
random_predict_image(model, '../DATA', transform)

Picked image: ./DATA\negative\person_patch_20240501_171445_827629_0.76.png
Actual label: negative (label=0)
Predicted label: 0.0
